In [ ]:
import pandas as pd
import requests
import os
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
#https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95
# Motor Vehicle Collisions - Crashes
#https://dev.socrata.com/foundry/data.cityofnewyork.us/h9gi-nx95

In [ ]:
endpoint = 'https://data.cityofnewyork.us/resource/h9gi-nx95.json'
response = requests.get(endpoint)
data = response.json()
df = pd.json_normalize(data)

In [ ]:
df.head()

In [ ]:
df = df.iloc[:,[0,4,5]]
df['crash_date'] = pd.to_datetime(df['crash_date'])


In [ ]:
df['crash_date'] = df['crash_date'].astype(str)
df['number_of_persons_injured'] = df['number_of_persons_injured'].astype(int)
df['number_of_persons_killed'] = df['number_of_persons_killed'].astype(int)
#df['purchase'].astype(str).astype(int)

In [ ]:
df.head()

In [ ]:
#Crear tabla en Amazon Redshift

In [ ]:
#Connect to the cluster
import redshift_connector

In [ ]:
conn = redshift_connector.connect(
     host=os.getenv('host'),
     database=os.getenv('database'),
     port=5439,
     user=os.getenv('user'),
     password=os.getenv('password'),
  )

In [ ]:
# Create a Cursor object
cursor = conn.cursor()

In [ ]:
#Create Table
create_table_sql = """CREATE TABLE guidogiantorno_coderhouse.collisions (
        crash_date varchar,
        number_of_persons_injured int,
        number_of_persons_killed int )
        """

cursor.execute(create_table_sql)

In [ ]:
#Insert Values

# Define the SQL statement to insert data into the table
insert_sql = """
    INSERT INTO guidogiantorno_coderhouse.collisions (crash_date, number_of_persons_injured, number_of_persons_killed)
    VALUES (%s, %s, %s)
"""

# Generate the values to be inserted as a list of tuples
values = [tuple(row) for row in df.values]

# Execute the SQL statement to insert data into the table
cursor.executemany(insert_sql, values)


In [ ]:
# Commit the changes to make them permanent
conn.commit()

In [ ]:
# Query a table using the Cursor
cursor.execute("select * from guidogiantorno_coderhouse.collisions")